# Code Documentation Task


In [ ]:
#| default_exp tutorials.copilot

In [ ]:
#| hide
from nbdev.showdoc import show_doc
from IPython.display import display, HTML
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
# importing dependencies
import re
import pandas as pd
from pandas import DataFrame
from pathlib import Path
from spannerlib import get_magic_session,Session,Span


In this tutorial we will show how to augment LLM context using formal rule based ie techniques, by presenting a copilot like code documentation pipeline.

Normally, when running a model like copilot in an IDE like pycharm or vs-code, it is given the context of the last $k$ files in our editor history as context for the code completion.

In this tutorial, we will see how we can use infromation extraction of spans of code based on the structure of the code to improve the context of the LLM.
First, let us get the `llm` and `format` ie function from the basic tutorial.

In [ ]:
# load openAI api key for env file

from dotenv import load_dotenv
load_dotenv('.env_dev')
import os
assert os.getenv('OPENAI_API_KEY') is not None

In [ ]:
from spannerlib.ie_func.basic import rgx_split
from functools import cache
import openai
from joblib import Memory
memory = Memory("cachedir", verbose=0)
client = openai.Client()

def str_to_messages (string_prompt):
    return [
        {
            'role': str(role).replace(': ',''),
            'content': str(content)
        } for role,content in rgx_split('system:\s|assistant:\s|user:\s', string_prompt.strip())
    ]
def messages_to_string(msgs):
    return ''.join([f"{msg['content']}" for msg in msgs])

def openai_chat(model, messages):
    respone = client.chat.completions.create(
        model=model,
        messages=messages,
        seed=42
    )
    return [dict(respone.choices[0].message)]

@memory.cache
def llm(model, question):
    q_msgs = str_to_messages(question)
    a_msgs = openai_chat(model, q_msgs)
    answer = messages_to_string(a_msgs)
    return answer

def llm_ie(model, question):
    return [llm(model, question)]




In [ ]:
def format_ie(f_string,*params):
    yield f_string.format(*params),

string_schema = lambda x: ([str]*x)

In [ ]:
sess.register('llm', llm_ie, [str,str],[str])
sess.register('format', format_ie, string_schema,[str])


In order to analyze the structure of the code, we will be using python's `ast` module.
We will write a very generic ie function that gets a piece of code, and an xpath query string returns the spans of all matches of the query over the ast of the given code.

To do so we will use the `pyastgrep` library that allows us to look for xpath matches in python ast's.
We will write a modified version of it's main function that returns Spans of the ast nodes.

In [ ]:
#TODO from here
# make previous tutorials export, but not register ie function in export
# clean the ast ie code and make a test
# move test to code test folder
# move covid samples to covid sample inputs dir
# make sure we have a tmp dir that is in gitignore that we can clone pandas to


# ! pip install ast2json json2ast
from jsonpath_ng import jsonpath, parse
import ast
from ast2json import ast2json,str2json
from json2ast import json2ast

In [ ]:
from astpath import search,convert_to_xml,find_in_ast

In [ ]:
from astpath.search import _tostring_factory
tostring = _tostring_factory()

In [ ]:
# ! pip install "astpath[xpath]"


In [ ]:
def print_ast(tree):
    print(ast.dump(tree, indent=2))


In [ ]:
! pip install pyastgrep

  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.8/219.8 kB 1.2 MB/s eta 0:00:0000:0100:01
Using cached pathspec-0.12.1-py3-none-any.whl (31 kB)


In [ ]:
from pyastgrep.search import search_python_files,Match

In [ ]:
matches = [m for m in search_python_files([Path('example_code.py')], '*/FunctionDef') if isinstance(m,Match)]
m = matches[0]

In [ ]:
m.ast_node.end_lineno

2

In [ ]:
m.path

Path('example_code.py')

In [ ]:
ast.unparse(m.ast_node)

'def f(x, y):\n    x + y'

In [ ]:
from functools import cache

def get_matching_nodes(path,expr):
    """get all ast nodes of the file path that matches the xpath expression expr"""
    for match in search_python_files([Path(path)], expr):
        if isinstance(match,Match):
            yield match.ast_node

@cache
def _get_lines(path):
    if not isinstance(path,Path):
        path = Path(path)
    return tuple(path.read_text().split('\n'))

def get_character_position(path, line_number, column_offset):
    """gets a character position from a line number and column offset"""
    lines = _get_lines(path)
    if line_number < 1 or line_number > len(lines):
        raise ValueError("Invalid line number")
    line = lines[line_number - 1]
    if column_offset < 0 or column_offset > len(line):
        raise ValueError("Invalid column offset")
    return sum(len(lines[i]) + 1 for i in range(line_number - 1)) + column_offset

def node_to_span(path,node):
    """given a node <node> of an ast from file <path>,
    returns the location of the node in the file as a Span object"""
    file_text = Path(path).read_text()
    start = get_character_position(path,node.lineno,node.col_offset)
    if hasattr(node,'end_lineno') and hasattr(node,'end_col_offset'):
        end = get_character_position(path,node.end_lineno,node.end_col_offset)
    else:
        end = start + len(ast.unparse(node))
    return Span(text,start,end)


In [ ]:
def ast_ie(path,expr):
    """returns a list of spans of the nodes that match the xpath expression <expr> in the file <path>"""
    for node in get_matching_nodes(path,expr):
        yield node_to_span(path,node)

In [ ]:
list(ast_ie('example_code.py','*/FunctionDef'))

[[@25a789,0,19) "def f(x,y)...", [@25a789,22,54) "def g(x,y)..."]

In [ ]:
convert_to_xml?

Signature: convert_to_xml(node, omit_docstrings=False, node_mappings=None)
Docstring: Convert supplied AST node to XML.
File:      ~/miniconda3/envs/span/lib/python3.11/site-packages/astpath/asts.py
Type:      function

In [ ]:
find_in_ast?

Signature:
find_in_ast(
    xml_ast,
    expr,
    query=<function _query_factory.<locals>.lxml_query at 0x12b2cf600>,
    node_mappings=None,
)
Docstring: Find items matching expression expr in an XML AST.
File:      ~/miniconda3/envs/span/lib/python3.11/site-packages/astpath/search.py
Type:      function

In [ ]:
print_ast(ast_tree)

Module(
  body=[
    FunctionDef(
      name='f',
      args=arguments(
        posonlyargs=[],
        args=[
          arg(arg='x'),
          arg(arg='y')],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        Expr(
          value=BinOp(
            left=Name(id='x', ctx=Load()),
            op=Add(),
            right=Name(id='y', ctx=Load())))],
      decorator_list=[]),
    FunctionDef(
      name='g',
      args=arguments(
        posonlyargs=[],
        args=[
          arg(arg='x'),
          arg(arg='y')],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        Return(
          value=BinOp(
            left=Call(
              func=Name(id='f', ctx=Load()),
              args=[
                Name(id='x', ctx=Load()),
                Name(id='y', ctx=Load())],
              keywords=[]),
            op=Pow(),
            right=Constant(value=2)))],
      decorator_list=[]),
    Expr(
      value=Call(
 

In [ ]:
dir(ast_tree)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_fields',
 'body',
 'type_ignores']

In [ ]:
%%spannerlog

model = 'gpt-3.5-turbo'
prompt = "user: sing it with me, love, what is it good for?"

TestLLM(answer)<-\
    llm($model,$prompt)->(answer)

?TestLLM(answer)


'?TestLLM(answer)'

,answer
0,Absolutely nothing!


Now let see how we can use spannerlib to build a simple pipeline that varries the LLM prompt in a data dependant way.
Imagine that we have a chatbot that we want to act differently based on the topic of conversation and the user preference.

To enable us to format prompts from data, we will use a prompt formatting function with a `printf` like syntax.
It is available in spannerlib's stdlib, but we will reproduce it here.


We will model our data as follows:

In [ ]:

# a binary relation that stores how the user prefers the formatted
user_answer_prefernce = pd.DataFrame([
    ('Bob', 'please answer in prose. Make sure you add references in the end like a bibliography.'),
    ('Joe', 'Prose is hard for me to read quickly. Format the answer in bullet points.'),
    ('Sally', 'Try to avoid complicated jargon and use simple language.'),
])

# relation that stores per topic style
topic_specific_style = pd.DataFrame([
    ('history', 'please answer in a narrative form, use shakespearing language and lots of examples'),
    ('science', 'Use mathematical notation and formulas to explain the concepts.'),
    ('hiphop', 'introduce the answer with a rap verse.'),
    ('other', 'Be polite and neutral in your answer. Avoid controversial topics.'),
])

sess.import_rel('UserAnswerPreference', user_answer_prefernce)
sess.import_rel('TopicSpecificStyle', topic_specific_style)

Now we will build our pipeline as follows:
* We will make an llm call to help us decide the topic of the question
* Based on the topic and the user, we will formulate the final prompt for the llm to answer.


In [ ]:
prompts = pd.DataFrame([
    ('topic_selection',
"""
system: Please select a topic from the following list: [history, science, hiphop, other]
based on the question provided by the user. You are only allwed to say the topic name, nothing else.

user: {}
"""),
    ('custom_style_prompt',
"""
system: Answer the question of the user in the following style:

topic specific style instructions: {}

user specific style instructions: {}

user: {}
"""
)
])
sess.import_rel('Prompts', prompts)
prompts

,0,1
0,topic_selection,system: Please select a topic from the follow...
1,custom_style_prompt,system: Answer the question of the user in th...


In [ ]:
%%spannerlog
?Prompts(prompt_id,prompt)

'?Prompts(prompt_id,prompt)'

,prompt_id,prompt
0,custom_style_prompt,system: Answer the question of the user in the following style: topic specific style instructions: {} user specific style instructions: {} user: {}
1,topic_selection,"system: Please select a topic from the following list: [history, science, hiphop, other] based on the question provided by the user. You are only allwed to say the topic name, nothing else. user: {}"


Now given a relation of user and question

In [ ]:
questions= pd.DataFrame([
    ('Bob', 'Who won the civil war?'),
    ('Joe', 'Who won the civil war?'),
    ('Sally', 'Who won the civil war?'),
    ('Bob', 'How do we measure the distance between stars?'),
    ('Joe', 'How do we measure the distance between stars?'),
    ('Sally', 'How do we measure the distance between stars?'),
    ('Bob', 'Who are the most well known rappers?'),
    ('Joe', 'Who are the most well known rappers?'),
    ('Sally', 'Who are the most well known rappers?'),
])
sess.import_rel('Questions', questions)

'?TopicPrompt(Q,P)'

,Q,P
0,How do we measure the distance between stars?,"system: Please select a topic from the following list: [history, science, hiphop, other] based on the question provided by the user. You are only allwed to say the topic name, nothing else. user: How do we measure the distance between stars?"
1,Who are the most well known rappers?,"system: Please select a topic from the following list: [history, science, hiphop, other] based on the question provided by the user. You are only allwed to say the topic name, nothing else. user: Who are the most well known rappers?"
2,Who won the civil war?,"system: Please select a topic from the following list: [history, science, hiphop, other] based on the question provided by the user. You are only allwed to say the topic name, nothing else. user: Who won the civil war?"


Now lets combine all of this into a pipeline.


In [ ]:
%%spannerlog
model = 'gpt-3.5-turbo'

TopicPrompt(Q,P)<-\
    Questions(user,Q),\
    Prompts('topic_selection',Template),\
    format(Template,Q)->(P)
    
?TopicPrompt(Q,P)

TopicSelection(Q,T)<-\
    TopicPrompt(Q,P),\
    llm($model,P)->(T)

?TopicSelection(Q,T)
StylePrompt(Q,P,Topic,user)<-\
    Questions(user,Q),\
    TopicSelection(Q,Topic),\
    UserAnswerPreference(user,UserStyle),\
    TopicSpecificStyle(Topic,TopicStyle),\
    Prompts('custom_style_prompt',promptTemplate),\
    format(promptTemplate,TopicStyle,UserStyle,Q)->(P)


Style_Based_QA(Q,Topic,user,P,A)<-\
    StylePrompt(Q,P,Topic,user),\
    llm($model,P)->(A)



And as you can see, we have a topic and user dependant LLM pipeline

In [ ]:
%%spannerlog
?Style_Based_QA(Q,Topic,user,Prompt,Answer)

'?Style_Based_QA(Q,Topic,user,Prompt,Answer)'

,Q,Topic,user,Prompt,Answer
0,How do we measure the distance between stars?,science,Bob,system: Answer the question of the user in the following style: topic specific style instructions: Use mathematical notation and formulas to explain the concepts. user specific style instructions: please answer in prose. Make sure you add references in the end like a bibliography. user: How do we measure the distance between stars?,"The distance between stars is measured in astronomical units (AU), parsecs (pc), or light-years (ly). 1. **Astronomical Unit (AU)**: An AU is the average distance between the Earth and the Sun, which is approximately 93 million miles or 150 million kilometers. It is more commonly used to measure distances within our solar system. 2. **Parsec (pc)**: A parsec is a unit of distance used in astronomy, defined as the distance at which an object would have a parallax angle of one arcsecond. One parsec is equal to about 3.26 light-years or 3.09 x 10^13 kilometers. 3. **Light-year (ly)**: A light-year is the distance that light travels in one year, which is approximately 9.46 trillion kilometers or about 5.88 trillion miles. To measure the distance to nearby stars, astronomers use a method called parallax. This involves observing a star from two different points in Earth's orbit around the Sun, which causes the star to appear to shift slightly against the background of more distant stars. By measuring the angle of this shift (the parallax angle), astronomers can calculate the distance to the star using trigonometry. For more distant stars, astronomers use other methods such as spectroscopic parallax, standard candles (like Cepheid variables), and the cosmic distance ladder which combines various distance measurement techniques to determine distances to faraway objects in the universe. References: 1. ""Astronomical unit"" - NASA, https://solarsystem.nasa.gov/solar-system/our-solar-system/overview/ 2. ""Parsec"" - European Space Agency, https://www.spacetelescope.org/about/general/parallax/ 3. ""Light-year"" - Space.com, https://www.space.com/15830-light-years.html 4. ""Parallax"" - European Southern Observatory, https://www.eso.org/public/usa/news/eso1718/"
1,How do we measure the distance between stars?,science,Joe,system: Answer the question of the user in the following style: topic specific style instructions: Use mathematical notation and formulas to explain the concepts. user specific style instructions: Prose is hard for me to read quickly. Format the answer in bullet points. user: How do we measure the distance between stars?,"- To measure the distance between stars, astronomers use a method called parallax, which is based on trigonometry. - Parallax involves observing a star from two different points in Earth's orbit and measuring the angle, known as the parallax angle, that the star appears to move. - The distance to the star can be calculated using the formula: distance = 1 / parallax angle. - The unit typically used for measuring stellar distances is the parsec (pc), where 1 parsec is equivalent to about 3.26 light years. - Other methods like photometric distance and spectroscopic parallax are also used to measure distances to stars that are much farther away."
2,How do we measure the distance between stars?,science,Sally,system: Answer the question of the user in the following style: topic specific style instructions: Use mathematical notation and formulas to explain the concepts. user specific style instructions: Try to avoid complicated jargon and use simple language. user: How do we measure the distance between stars?,"One common way to measure the distance between stars is through parallax. Parallax is the apparent shift in position of an object when viewed from two different points. The parallax angle, denoted by p, is the angle formed between a line from the observer to the star at two different times when Earth is on opposite sides of its orbit. The distance to the star, denoted by d, can be calculate